# Deep learning: classifying dog breeds

This project consists of setting up infrastructure to manage the lifecycle of a machine learning model, using AWS resources such as Sagemaker and EC2 to create, train and deploy a pretrained model that can classify images from a dataset containing different dog breeds.

The focus of this proyect is to set up infrastructure that supports the functioning of machine learning models, therefore other tools to monitor and test the model's performance will be used in addition to the activities mentioned above.

In [ ]:
#Install needed dependencies
!pip install smdebug
#!pip install sagemaker #Sagemaker instances already have this library

In [4]:
#Import libraries
import sagemaker
import boto3
import os
from torchvision.datasets import StanfordCars
#For Hyperparameter tuning
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)
from torchvision import transforms
from sagemaker.pytorch import PyTorch #Estimator

#For debugging and profiling
from sagemaker.debugger import Rule, DebuggerHookConfig, ProfilerRule, rule_configs
from sagemaker.debugger import ProfilerConfig, FrameworkProfile


from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts

import IPython




session = sagemaker.Session()
bucket = session.default_bucket()
role = sagemaker.get_execution_role()
region = session.region_name

## Dataset

This project uses the Stanford Cars dataset, which according to its creators:
> Contains 16,185 images of 196 classes of cars. The data is split into 8,144 training images and 8,041 testing images, where each class has been split roughly in a 50-50 split. Classes are typically at the level of Make, Model, Year, e.g. 2012 Tesla Model S or 2012 BMW M3 coupe.

In [ ]:
# Command to download and unzip data

prefix = "StanfordCars/"

local_dir = 'data'
StanfordCars.mirrors = ["https://sagemaker-sample-files.s3.amazonaws.com/datasets/image/StanfordCars/"]
imagae_dataset = StanfordCars(
    local_dir,
    download=True,
    transform=transforms.Compose(
        [transforms.ToTensor(),
        transforms.Resize(3,244,244)]
    )
)

inputs = session.upload_data(path="data", bucket=bucket, key_prefix=prefix)
print("input spec (in this case, just an S3 path): {}".format(inputs))

#### For curious readers:

Datasets included in the pytorch library are listed in this [link](https://pytorch.org/vision/stable/generated/torchvision.datasets.StanfordCars.html#torchvision.datasets.StanfordCars) <br><br>
For the STANFORD CARS dataset, which is used for this project, documentation can be consulted in the [Pytorch website](https://pytorch.org/vision/stable/generated/torchvision.datasets.StanfordCars.html#torchvision.datasets.StanfordCars). Moreover, the original documentation on this dataset can be found [here](https://ai.stanford.edu/~jkrause/cars/car_dataset.html)

## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [ ]:
#Declare hyperparameters that will be tuned
hyperparameter_ranges = {
    "lr": ContinuousParameter(0.001, 0.1),
    "batch-size": CategoricalParameter([32, 64, 128, 256, 512]),
    'epochs': CategoricalParameter([5,10,15,30,60])
}

In [ ]:
estimator = PyTorch(
    entry_point='hpo.py',
    role=role,
    py_version='py39',
    framework_version='1.13',
    instance_count=1,
    instance_type='ml.m5.xlarge'
)

#The training script uses cross entropy loss function, as this is a multiclass classification problem
objective_metric_name = "average test loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "average test loss", "Regex": "Test set: Average loss: ([0-9\\.]+)"}]

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=4,
    max_parallel_jobs=2,
    objective_type=objective_type
)

In [ ]:
#Fit your HP Tuner
tuner.fit({'training': inputs}, wait=True)

In [ ]:
best_estimator = tuner.best_estimator

#Get the hyperparameters of the best trained model
best_estimator.hyperparameters()

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [ ]:
# TODO: Set up debugging and profiling rules and hooks
rules = [
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
    ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
]
hook_config = DebuggerHookConfig(
    hook_parameters={'train.save_interval': "100", "eval.save_interval": "10"})
profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=10)
)

In [ ]:

#TODO: get hyperparameters from hyperparameter tuning job above
best_hyperparameters = #dictionary


estimator = PyTorch(
    entry_point='train_model.py',
    role=role,
    py_version='py39',
    framework_version='1.13',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    hyperparameters=best_hyperparameters,
    rules=rules,
    debugger_hook_config=hook_config,
    profiler_config=profiler_config,
)

In [ ]:
estimator.fit({'training': inputs})

In [ ]:
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob

tj = TrainingJob(training_job_name, region)
tj.wait_for_sys_profiling_data_to_be_available()

In [ ]:
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts

system_metrics_reader = tj.get_systems_metrics_reader()
system_metrics_reader.refresh_event_file_list()

view_timeline_charts = TimelineCharts(
    system_metrics_reader,
    framework_metrics_reader=None,
    select_dimensions=["CPU", "GPU"],
    select_events=["total"],
)

Profiler analysis code taken from [aws github](https://github.com/aws/amazon-sagemaker-examples/blob/main/sagemaker-debugger/pytorch_profiling/pt-resnet-profiling-single-gpu-single-node.ipynb)

In [ ]:
from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys

trial = create_trial(estimator.latest_job_debugger_artifacts_path())

print(trial.tensor_names())#We cn print all the tensors that were recorded to know what we want to plot

print(len(trial.tensor("XEntropyLoss_output_0").steps(mode=ModeKeys.TRAIN)))
print(len(trial.tensor("XEntropyLoss_output_0").steps(mode=ModeKeys.EVAL)))


def get_data(trial, tname, mode):
    tensor = trial.tensor(tname)
    steps = tensor.steps(mode=mode)
    vals = []
    for s in steps:
        vals.append(tensor.value(s, mode=mode))
    return steps, vals
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import host_subplot


def plot_tensor(trial, tensor_name):

    steps_train, vals_train = get_data(trial, tensor_name, mode=ModeKeys.TRAIN)
    print("loaded TRAIN data")
    steps_eval, vals_eval = get_data(trial, tensor_name, mode=ModeKeys.EVAL)
    print("loaded EVAL data")

    fig = plt.figure(figsize=(10, 7))
    host = host_subplot(111)

    par = host.twiny()

    host.set_xlabel("Steps (TRAIN)")
    par.set_xlabel("Steps (EVAL)")
    host.set_ylabel(tensor_name)

    (p1,) = host.plot(steps_train, vals_train, label=tensor_name)
    print("completed TRAIN plot")
    (p2,) = par.plot(steps_eval, vals_eval, label="val_" + tensor_name)
    print("completed EVAL plot")
    leg = plt.legend()

    host.xaxis.get_label().set_color(p1.get_color())
    leg.texts[0].set_color(p1.get_color())

    par.xaxis.get_label().set_color(p2.get_color())
    leg.texts[1].set_color(p2.get_color())

    plt.ylabel(tensor_name)

    plt.show()
    
plot_tensor(trial, "XEntropyLoss_output_0")
  

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output
rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"
! aws s3 ls {rule_output_path} --recursive
! aws s3 cp {rule_output_path} ./ --recursive
# get the autogenerated folder name of profiler report
profiler_report_name = [
    rule["RuleConfigurationName"]
    for rule in estimator.latest_training_job.rule_job_summary()
    if "Profiler" in rule["RuleConfigurationName"]
][0]
import IPython

IPython.display.HTML(filename=profiler_report_name + "/profiler-output/profiler-report.html")

## Model Deploying

In [ ]:
# TODO: Deploy your model to an endpoint

predictor=estimator.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge') # TODO: Add your deployment configuration like instance type and number of instances

In [ ]:
# TODO: Run an prediction on the endpoint
sample_idx = torch.randint(len(image_dataset), size=(1,)).item()
image, label = image_dataset[sample_idx]



response = predictor.predict(image)
print(response)

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()

In [ ]:
https://pytorch.org/vision/stable/transforms.html

In [ ]:
transformar la imagen en tensor para que sea el formato que pytorch espera.
Y si no funciona, usar las transformaciones en los scripts de entrenamiento para obtener una imagen de 244*244*3
(solamente cambiar el ancho y el largo, ya que todas son a color)

hacer estas mismas transformaciones cuando se mande a llamar el endpoint con la imagen para probar la prediccion